In [4]:
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D ,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K 
import cv2
import numpy as np
from keras.preprocessing import image 

In [5]:
train_data_dir = 'Training' 
test_data_dir = 'Validation'
# nb_train_samples = 900 
# nb_validation_samples = 100
epochs = 10
batch_size = 8
img_width, img_height = 224, 224

In [6]:
if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3) 

In [7]:
train_datagen = ImageDataGenerator( 
    rescale=1. / 255, 
    shear_range=0.2, 
    zoom_range=[-1.0,1.0], 
    horizontal_flip=True,
    width_shift_range=[-200,200],
    height_shift_range=0.5,
    rotation_range=40,
    brightness_range=[0.2,1.0]) 
  
test_datagen = ImageDataGenerator(rescale=1. / 255) 

In [8]:
train_generator = train_datagen.flow_from_directory( 
    train_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory( 
    test_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode='categorical')

Found 2522 images belonging to 3 classes.
Found 255 images belonging to 3 classes.


In [9]:
model = Sequential()

#bottle neck model
model.add(Conv2D(filters=64 ,padding='same',input_shape=input_shape , kernel_size=(4, 4),activation='relu'))
#model.add(Conv2D(filters=64, padding='same',kernel_size=(4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(filters=128 ,padding='same', kernel_size=(4,4),activation='relu'))
#model.add(Conv2D(filters=128,padding='same', kernel_size=(4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(filters=256  ,padding='same', kernel_size=(4,4),activation='relu'))
#model.add(Conv2D(filters=256,padding='same', kernel_size=(4,4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# model.add(Conv2D(filters=512  ,padding='same', kernel_size=(4,4),activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

#Flatten the input data
model.add(Flatten())

#Neural Network
model.add(Dense(output_dim=128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(output_dim=64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(output_dim=32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#Softmax classifier
model.add(Dense(output_dim=3, activation='softmax'))

model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      3136      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     131200    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 128)       512       
__________

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=32)`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=3)`


In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, 
    steps_per_epoch = 2522  // batch_size, 
    epochs = epochs, validation_data = validation_generator,
    validation_steps = 255  // batch_size) 

Epoch 1/10
 10/315 [..............................] - ETA: 18:57 - loss: 2.0919 - acc: 0.3375

In [ ]:
model.save('sach_pandit_binu.h5')